# Fire Detection Dataset Preprocessing

This notebook handles:
1. Resizing images and masks to 512x512
2. Converting segmentation masks to YOLO format bounding boxes (handling multiple fires)
3. Saving annotations in the required format (including empty annotations)

In [9]:
import os
import random
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [2]:
# Configure paths
INPUT_PATH = Path('../dataset')
IMAGE_PATH = INPUT_PATH / 'Images'
MASK_PATH = INPUT_PATH / 'Masks'
OUTPUT_PATH = Path('../processed_dataset')

# Create output directories
OUTPUT_PATH.mkdir(exist_ok=True)
(OUTPUT_PATH / 'images').mkdir(exist_ok=True)
(OUTPUT_PATH / 'masks').mkdir(exist_ok=True)
(OUTPUT_PATH / 'labels').mkdir(exist_ok=True)

In [3]:
def mask_to_bboxes(mask, min_area=10):
    """Convert binary mask to multiple YOLO format bounding boxes.
    
    Args:
        mask: Binary mask image
        min_area: Minimum contour area to consider as a valid fire region
        
    Returns:
        List of bounding boxes in YOLO format [x_center, y_center, width, height]
    """
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bboxes = []
    image_h, image_w = mask.shape
    for contour in contours:
        # Filter out small contours
        if cv2.contourArea(contour) < min_area:
            continue
            
        x, y, w, h = cv2.boundingRect(contour)
        
        # Convert to YOLO format (normalized coordinates)
        x_center = (x + w/2) / image_w
        y_center = (y + h/2) / image_h
        width = w / image_w
        height = h / image_h
        
        # Ensure values are within [0, 1]
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))
        
        bboxes.append([x_center, y_center, width, height])
    
    return bboxes

In [4]:
def process_image_and_mask(image_path, mask_path, target_size=(512, 512)):
    """Process single image and mask pair."""
    # Read image and mask
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
        
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError(f"Could not read mask: {mask_path}")
    
    # Resize image and mask
    image_resized = cv2.resize(image, target_size)
    mask_resized = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
    
    # Convert mask to binary
    # _, mask_binary = cv2.threshold(mask_resized, 127, 255, cv2.THRESH_BINARY)
    
    # Get bounding boxes in YOLO format
    bboxes = mask_to_bboxes(mask_resized)
    
    return image_resized, mask_resized, bboxes

In [5]:
image_path = '../dataset/Images/image_0.jpg'
mask_path = '../dataset/Masks/image_0.png'
res = process_image_and_mask(image_path, mask_path)

In [6]:
res[2]

[[0.7783203125, 0.2158203125, 0.025390625, 0.033203125],
 [0.9404296875, 0.201171875, 0.025390625, 0.0234375],
 [0.5849609375, 0.1982421875, 0.087890625, 0.119140625]]

In [7]:
def process_dataset():
    """Process all images and masks in the dataset."""
    image_files = sorted(list(IMAGE_PATH.glob('*.jpg')))
    processed_count = 0
    error_count = 0
    
    for image_path in tqdm(image_files):
        try:
            # Get corresponding mask path
            mask_path = MASK_PATH / f"{image_path.stem}.png"
            
            if not mask_path.exists():
                print(f"Warning: No mask found for {image_path}")
                # Create an empty label file
                with open(OUTPUT_PATH / 'labels' / f"{image_path.stem}.txt", 'w') as f:
                    pass
                continue
            
            # Process image and mask
            image_resized, mask_resized, bboxes = process_image_and_mask(image_path, mask_path)
            
            # Save processed files
            cv2.imwrite(str(OUTPUT_PATH / 'images' / image_path.name), image_resized)
            cv2.imwrite(str(OUTPUT_PATH / 'masks' / mask_path.name), mask_resized)
            
            # Save YOLO format annotations (even if empty)
            with open(OUTPUT_PATH / 'labels' / f"{image_path.stem}.txt", 'w') as f:
                for bbox in bboxes:
                    f.write(f"0 {' '.join(map(lambda x: f'{x:.6f}', bbox))}\n")
            
            processed_count += 1
            
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
            error_count += 1
            
    print(f"\nProcessing complete:")
    print(f"Successfully processed: {processed_count} images")
    print(f"Errors encountered: {error_count} images")

In [8]:
# Process the dataset
process_dataset()

100%|██████████| 2003/2003 [01:54<00:00, 17.47it/s]


Processing complete:
Successfully processed: 2003 images
Errors encountered: 0 images


In [10]:
def verify_processing():
    """Verify the processed dataset."""
    print("Verifying processed dataset...")
    
    # Check directories
    for dir_name in ['images', 'masks', 'labels']:
        dir_path = OUTPUT_PATH / dir_name
        file_count = len(list(dir_path.glob('*')))
        print(f"{dir_name}: {file_count} files")
    
    # Verify image sizes
    image_files = list((OUTPUT_PATH / 'images').glob('*.jpg'))
    if image_files:
        sample_image = cv2.imread(str(image_files[0]))
        print(f"\nImage size: {sample_image.shape}")
        
    # Check label format
    label_files = list((OUTPUT_PATH / 'labels').glob('*.txt'))
    if label_files:
        print("\nSample labels:")
        for label_file in random.sample(label_files, min(3, len(label_files))):
            with open(label_file, 'r') as f:
                content = f.read().strip()
                print(f"{label_file.name}: {content if content else 'empty'}")

In [11]:
# Verify the processing
verify_processing()

Verifying processed dataset...
images: 2003 files
masks: 2003 files
labels: 2003 files

Image size: (512, 512, 3)

Sample labels:
image_1593.txt: 0 0.542969 0.818359 0.042969 0.085938
0 0.333984 0.651367 0.007812 0.013672
0 0.317383 0.614258 0.033203 0.150391
0 0.006836 0.144531 0.013672 0.042969
image_381.txt: 0 0.541992 0.084961 0.013672 0.021484
0 0.569336 0.065430 0.013672 0.037109
0 0.566406 0.031250 0.011719 0.019531
0 0.554688 0.025391 0.011719 0.015625
image_1859.txt: 0 0.073242 0.994141 0.009766 0.011719
0 0.111328 0.969727 0.015625 0.025391
0 0.502930 0.887695 0.060547 0.119141
0 0.170898 0.441406 0.013672 0.035156
0 0.448242 0.353516 0.013672 0.042969
0 0.279297 0.220703 0.007812 0.023438
